In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import csv
import seaborn as sns
import string

In [ ]:
#Defining function to clean up dups from key field 'Make'
def func(x):
    punc = '''-!()[]{};:'",<>./?@#$%^&*~'''
    for ele in str(x):
        if ele in punc:
            x = str(x).replace(ele, "")
    x = str(x).title()
    x = x.replace(" Inc","")
    x = x.replace(" Corp","")
    x = x.replace(" Co","")
    x = x.replace("Airborne Windsports","Airborne_Windsports")
    x = x.replace("Airborne Windsport","Airborne_Windsports")
    x = x.replace("Airborne_Windsports Ltd","Airborne_Windsports")
    x = x.replace("Airborne_Windsports Pty Ltd","Airborne_Windsports")
    x = x.replace("Air Borne Windsports Pty Ltd","Airborne_Windsports")
    x = x.replace("Airbus Industrie","Airbus")
    x = x.replace("Airbuss","Airbus")
    x = x.replace("Boeingmpany","Boeing")
    x = x.replace("The Boeingmpany","Boeing")
    x = x.replace("The Boeing","Boeing")
    x = x.replace("Brantley","Brantly")
    x = x.replace("Mcdonnell Douglasoration","Mcdonnell_Douglas")
    x = x.replace("Mcdonald Douglas","Mcdonnell_Douglas")
    x = x.replace("Mcdonnell-Douglas","Mcdonnell_Douglas")
    x = x.replace("Mcdonnell Douglas Aircraft","Mcdonnell_Douglas")
    x = x.replace("Douglas","Mcdonnell_Douglas")
    x = x.replace("Mcdonnell Mcdonnell_Douglas","Mcdonnell_Douglas")
    x = x.replace("Saabscania","Saab_Scania")
    x = x.replace("Saab-Scania Ab Saab","Saab_Scania")
    x = x.replace("Saabscania Ab Saab","Saab_Scania")
    x = x.replace("Saab-Scania","Saab_Scania")
    x = x.replace("Saab_Scania Ab Saab","Saab_Scania")
    x = x.replace("Waco","Waco_Classic_Aircraft")
    x = x.replace("Classic Aircraft","Waco_Classic_Aircraft")
    x = x.replace("Waco Classic Aircraft","Waco_Classic_Aircraft")
    x = x.replace("Waco_Classic_Aircraft Waco_Classic_Aircraft","Waco_Classic_Aircraft")
    x = x.replace("Schweitzer","Schweizer")
    x = x.replace("Schewizer","Schweizer")
    x = x.replace("Pm Aviation Ltd","Pm Aviation")
    x = x.replace("Schleicher Alexander Gmbh ","Schleicher Alexander Gmbh")
       
    return x

In [ ]:
#Cleaning up dups from key fields and downstream dependency & imputing weather condition field for further analysis
df = pd.read_csv('AviationDataCleaned.csv',low_memory=False)
df['Make'] = df['Make'].apply(func)
df['Model'] = df['Model'].apply(func)
df['Make and Model']=df['Make and Model']= df['Make']+"-"+df['Model'] 
df['Weather.Condition'] = df['Weather.Condition'].fillna('UNK').apply(lambda x: str(x).upper())
df['Weather.Condition'].unique()

In [ ]:
#inspecting the Make column to make sure it is cleaned up appropriately
make_list=list(df['Make'].unique())
make_list.sort()
print(make_list)

In [ ]:
#coming up with makes with at least 10 records to consider as major makes to narrow down data set
major_index = df['Make'].value_counts(ascending=False)>10
major_makes_dict={'makes':list(major_index.index),'bools':list(major_index.values)}
major_makes = pd.DataFrame(major_makes_dict)
major_makes_list=major_makes.loc[major_makes['bools']==True,'makes']
df_major=df[df['Make'].isin(major_makes_list)]

In [ ]:
#inspecting the size of narrowed down data set
df[df['Make'].isin(major_makes_list)].shape

In [ ]:
type(df_major.groupby('Make').agg(lambda x: np.mean(x)))

In [ ]:
#Coming up with makes that are considered safest based on the safety metric aka with the highest average percentage of passengers with no or minor injury
#Then Slicing the dataset with the these makes and a known weather condition for further analysis  
df_major_make_mean = df_major.groupby('Make').mean().sort_values(by=['Minor/NonInj_Percentage'],axis=0,ascending=False)
safest_makes=list(df_major_make_mean.head(15).index)
df_safest = df[df['Make'].isin(safest_makes)]

df_safest_makes=df_safest[df_safest['Weather.Condition'].isin(['IMC','VMC'])]
df_safest_makes[['Make']]=df_safest_makes[['Make']].astype('category')
df_safest_makes.info()

In [ ]:
#just inspecting the stats of these makes 
df_major_make_mean.head(15)

In [ ]:
#plot to demonstrate the safest makes and their safety metrics
fig, ax = plt.subplots(figsize = (20,10))
sns.barplot(x='Make',y='Minor/NonInj_Percentage',data=df_safest,order=list(safest_makes))
ax.set_title('Aircraft Makes with the Highest Safety Metrics', fontsize = 30 )
ax.set_xlabel('Makes of Aircrafts', fontsize = 20)
ax.set_ylabel('Average % of Passengers with No or Minor Injuries', fontsize = 14)
ax.tick_params(axis = 'x', rotation = 90, pad = -400, labelsize=20,colors='white')
ax.axhline(y=85.797161, c ='b', linewidth = 2, linestyle = '--')
plt.tight_layout()
plt.show()


In [ ]:
#making sure no percentages are negative....
df_safest_makes[df_safest_makes['Minor/NonInj_Percentage']<0]

In [ ]:
df1=df_safest_makes[df_safest_makes['Make']=='Waco_Classic_Aircraft']
df2=df1[df1['Weather.Condition']=='IMC']
df2

In [ ]:
df3=df_safest_makes[df_safest_makes['Make']=='Dassaultbreguet']
df4=df3[df3['Weather.Condition']=='IMC']
df4

In [ ]:
df5=df_safest_makes[df_safest_makes['Make']=='Grummanschweizer']
df6=df5[df5['Weather.Condition']=='IMC']
df6

In [ ]:
df7=df_safest_makes[df_safest_makes['Make']=='Evektoraerotechnik As']
df8=df7[df7['Weather.Condition']=='IMC']
df8

In [ ]:
index_to_drop=[list(df2.index)[0],list(df2.index)[1],list(df4.index)[0],list(df4.index)[1],list(df4.index)[2],list(df6.index)[0],list(df8.index)[0]]
df_safest_makes=df_safest_makes.drop(index_to_drop)

In [ ]:
weather_map={'IMC':'Instrument meteorological conditions - less visibility','VMC':'Visual meteorological conditions - normal visibility'}
df_safest_makes=df_safest_makes.replace({'Weather.Condition':weather_map})

In [ ]:
#plot to analyze the distribution of safety metric for the safest makes based on weather condition
fig, ax = plt.subplots(figsize = (25,10))
sns.violinplot(x='Make',y='Minor/NonInj_Percentage',data = df_safest_makes,order=list(safest_makes),hue='Weather.Condition', ax = ax, split = True, cut=0)
ax.set_title('Distributions of Safety Metrics by Weather Conditions', fontsize = 29, loc='center')
ax.set_xlabel('Makes of Aircrafts', fontsize = 20)
ax.set_ylabel('% of Passengers with No or Minor Injuries', fontsize = 20)
ax.tick_params(axis = 'x', rotation = 70, labelsize=20,colors='black')
plt.legend()
ax.legend(loc = 'upper right',bbox_to_anchor=(1, 1.07))
plt.show()

In [ ]:
#Used to inpsect makes with no IMC weather hue distributions to confirm they are not suitable for recommendation. 
#These are generally small makes that are more impacted by adverse weather conditions and therefore less versatile and profitable
df[df['Make']=='American Legend Aircraft']['Total_passengers'].describe()

In [ ]:
# coming up with the list of recommended makes that are more weather resilient and predictable in safety metric (less dispersed in distribution)
recommended_makes=['Bombardier','Saab_Scania','Boeing']
#slicing the data set with recommeded makes for specific model recommendations
len(df[df['Make'].isin(recommended_makes)])

In [ ]:
#Coming up with specific recommended models with 100% minor or no injury rates that holds at least 10 passengers
df_subset_makenmodel_mean = df[df['Make'].isin(recommended_makes)].groupby('Make and Model').mean()
df_recommended_makemodels=df_subset_makenmodel_mean[df_subset_makenmodel_mean['Minor/NonInj_Percentage']==100]
df_recommended_makenmodels=df_recommended_makemodels[df_recommended_makemodels['Total_passengers']>10]
recommended_makenmodels=list(df_recommended_makenmodels.index)
recommended_makenmodels

In [ ]:
#Creating dataset filtered for recommended makes and models
df_recommended_makes=df[df['Make'].isin(recommended_makes)]
df_recommended_makenmodels=df[df['Make and Model'].isin(recommended_makenmodels)]

In [ ]:
#Outputting recommedations as csvs
df_recommended_makenmodels.to_csv('AviationDataCleaned_RecommendedModel.csv')
df_recommended_makes.to_csv('AviationDataCleaned_RecommendedMakes.csv')

### Regional Analysis

In [ ]:
# Bring in CSV that Sunny's notebook outputs
df = pd.read_csv('AviationDataCleaned_RecommendedMakes.csv',index_col=0)

# Display charts inline for easy reading
%matplotlib inline

#### Rename an unnamed column that was breaking Tableau import later on

In [ ]:
df.rename(columns={"Unnamed: 0":"F1"},inplace=True)

#### Create a subset of the DF that has only US accidents

In [ ]:
df_us = df[df['Country'] == 'United States']
df_us.reset_index(drop=True, inplace=True)

# Create a copy of the DF so I don't destroy the original on accident downstream
df_us = df_us.copy()

### Create City, State, and Region columns based on the Location column 

* First we need a list of valid state codes to verify that the location has a valid US states
* Next we'll create a state-region map that will assign a region to the corresponding State

In [ ]:
states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

state_region_map = {
    'ME': 'Northeast', 'NH': 'Northeast', 'VT': 'Northeast', 'MA': 'Northeast',
    'RI': 'Northeast', 'CT': 'Northeast', 'NY': 'Northeast', 'PA': 'Northeast', 'NJ': 'Northeast',
    'WI': 'Midwest', 'MI': 'Midwest', 'IL': 'Midwest', 'IN': 'Midwest', 'OH': 'Midwest',
    'ND': 'Midwest', 'SD': 'Midwest', 'NE': 'Midwest', 'KS': 'Midwest', 'MN': 'Midwest', 'IA': 'Midwest', 'MO': 'Midwest',
    'DE': 'South', 'MD': 'South', 'DC': 'South', 'VA': 'South', 'WV': 'South', 'NC': 'South', 'SC': 'South', 
    'GA': 'South', 'FL': 'South', 'KY': 'South', 'TN': 'South', 'MS': 'South', 'AL': 'South', 'OK': 'South', 
    'TX': 'South', 'AR': 'South', 'LA': 'South',
    'ID': 'West', 'MT': 'West', 'WY': 'West', 'NV': 'West', 'UT': 'West', 'CO': 'West', 'AZ': 'West', 
    'NM': 'West', 'AK': 'West', 'WA': 'West', 'OR': 'West', 'CA': 'West', 'HI': 'West'
}

def city_state(location):
    if pd.notna(location):
        location = location.strip()
        state = location[-2:].upper()
        city = location[:-4].strip().title()
        if state in states:
            return city, state
        else:
            return location, "No State"
    else:
        return np.nan, np.nan

df_us[['City', 'State']] = df_us['Location'].apply(city_state).apply(pd.Series)

df_us['Region'] = df_us['State'].apply(lambda x: state_region_map.get(x))

In [ ]:
df_us = df_us[(df_us['State'] != 'No State') & (df_us['Year']>=1984)]

# Average Minor Injury % by Year & Region
grouped_data = df_us.groupby(['Year', 'Region'])['Minor/NonInj_Percentage'].mean().reset_index(name='Count')

pivot_data = grouped_data.pivot(index='Year', columns='Region', values='Count')

pivot_data.plot(kind='line')  # Use 'bar' for bar plot
plt.title('Total Count of Rows Year over Year by State')
plt.xlabel('Year')
plt.ylabel('Count')

plt.show()

In [ ]:
col_selector = ['Region', 'Minor/NonInj_Percentage', 'Serious_Inj_Percentage', 'Fatality_Pecentage']
total_new = df_us[col_selector].groupby('Region').mean()
# total_new['Total'] = totals

total_new

In [ ]:
melted_df = total_new.melt(var_name='Category', value_name='Value')

melted_df = total_new.melt(var_name='Category', value_name='Value')

# Create a bar plot with catplot
sns.catplot(x='Category', y='Value', hue='Category', data=melted_df, kind='bar')

# Adding labels and title
plt.xlabel('X-axis Label')
plt.xticks(rotation=45)
plt.ylabel('Y-axis Label')
plt.title('Side-by-Side Bar Chart')

# Display the plot

plt.show()

In [ ]:
plt.bar(total_new.index, total_new['Fatality_Pecentage'])

plt.bar(total_new.index, total_new['Serious_Inj_Percentage'], label='Serious Injury Percentage', bottom=total_new['Fatality_Pecentage'])

### Export this finalized CSV to upload to Tableau 

In [ ]:

df_us.to_csv('US_Accidents.csv')

## Pricing Analysis

In [ ]:
#import pricing mapping

price_df = pd.read_csv('Price_df.csv',low_memory=False)
price_df

In [ ]:
#Removing daptapoints with no values

price_df_clean = price_df[price_df['Passengers'] != 0]
price_df_clean


In [ ]:
#making a calculation column

ppp = (price_df_clean['Price'] / price_df_clean['Passengers'])

#reformatting the data so it looks better / not in scientific notation

pd.options.display.float_format = '{:.0f}'.format
    
price_df_clean['Price Per Passenger'] = ppp
price_df_clean['Price Per Passenger'].astype('category')
price_df_clean['Passengers'].astype('category')
price_df_clean

#Separating out the size tranches

best_high = price_df_clean[price_df_clean['Passengers'] >= 100]
best_high = best_high[best_high['Price Per Passenger'] <= 400000]
best_high_sorted = best_high.sort_values('Price Per Passenger')

best_mid = (price_df_clean[price_df_clean['Passengers'] <= 100]) 
best_mid = best_mid[best_mid['Passengers'] >= 50]
best_mid = best_mid[best_mid['Price Per Passenger'] <= 100000000]
best_mid_sorted = best_mid.sort_values('Price Per Passenger')


best_low = price_df_clean[price_df_clean['Passengers'] <= 50]
best_low = best_low[best_low['Price Per Passenger'] <= 100000000]
best_low_sorted = best_low.sort_values('Price Per Passenger')


best_low_sorted

In [ ]:
title = 'Cost Effectiveness of Large Safe Aircraft'
Model_Label = 'Model'
PPP_label = 'Price Per Seat'

tackle_figure, ax = plt.subplots(figsize=(15, 6))

Model = best_high_sorted['Model']
PPP = best_high_sorted['Price Per Passenger']

ax.bar(Model,PPP)
ax.set_xlabel(Model_Label)
ax.set_ylabel(PPP_label)
ax.set_title(title)


plt.tight_layout()
plt.show()

In [ ]:
title = 'Cost Effectiveness of Mid Safe Aircraft'
Model_Label = 'Model'
PPP_label = 'Price Per Seat'

Plane_figure, ax = plt.subplots(figsize=(15, 6))

Model = best_mid_sorted['Model']
PPP = best_mid_sorted['Price Per Passenger']
xtick = 10000
ax.bar(Model,PPP)
ax.set_xlabel(Model_Label)
ax.set_ylabel(PPP_label)
ax.set_title(title)
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.tight_layout()
plt.show()
ax.get_yticks()

In [ ]:
title = 'Cost Effectiveness of Small Safe Aircraft'
Model_Label = 'Model'
PPP_label = 'Price Per Seat'

small_figure, ax = plt.subplots(figsize=(15, 6))

Model = best_low_sorted['Model']
PPP = best_low_sorted['Price Per Passenger']

ax.bar(Model,PPP)
ax.set_xlabel(Model_Label)
ax.set_ylabel(PPP_label)
ax.set_title(title)
ax.get_yaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.tight_layout()
plt.show()